# Spark Streaming
This notebook takes in the `gemini-feed` Kafka topic and produces to the `spark.out` topic a feed which includes the order price volume ratio and bid/ask liquidity for BTC.

In [2]:
!pip install kafka-python

In [3]:
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] =  '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.3 pyspark-shell'

In [4]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils  
from kafka import KafkaProducer
import json
import time

In [5]:
producer = KafkaProducer(bootstrap_servers='kafka-node:9092')

In [6]:
sc = SparkContext(appName="PythonSparkStreamingKafka_RM_01")  
sc.setLogLevel("ERROR") 
ssc = StreamingContext(sc, 60)  

num_streams=5 
kafkaStreams = [KafkaUtils.createStream(ssc,"kafka-node:2181","spark-streaming",{"gemini-feed":1}) for i in range(num_streams)]
unified_stream = ssc.union(*kafkaStreams)


In [7]:
kafkaStream = KafkaUtils.createStream(ssc, 'kafka-node:2181', 'spark-streaming', {'gemini-feed':1})  

In [8]:
parsed = kafkaStream.map(lambda v: json.loads(v[1]))

In [9]:
def handler(message):
    records = message.collect()
    for record in records:
        output = {}
        output['type'] = record[0]
        output['value'] = record[1]
        output['timestamp'] = int(time.time())
        producer.send('spark.out', bytes(json.dumps(output), 'utf-8'))
        producer.flush()

In [10]:
def price_volume(x):
    price = float(x['price'])
    remaining = float(x['remaining'])
    x['price_volume'] = price * remaining
    return (x['side'], x['price_volume'])

Get price/volume ratio

In [11]:
parsed_pv = parsed.map(lambda x: price_volume(x))

Group by side (bid or ask)

In [12]:
grouped = parsed_pv.reduceByKey(lambda accum, n: accum + n)

In [13]:
grouped_sorted = grouped.transform(lambda rdd: rdd.sortByKey())

In [14]:
grouped_sorted.foreachRDD(handler)

Generate a ratio of the price_volume of ask to bids. So, if ask volume is 4000 and bid volume is 1000 then the ratio is 4. This suggests ask (sell) side liquidity greatly exceeds buy side.

In [15]:
results = grouped_sorted.map(lambda x: ('price_volume', x[1]))

In [16]:
ratio = results.reduceByKey(lambda x, y: x/y)

In [17]:
ratio.foreachRDD(handler)

Print results

In [18]:
grouped_sorted.pprint()
ratio.pprint()

In [19]:
ssc.start()

-------------------------------------------
Time: 2019-05-25 06:43:00
-------------------------------------------
('ask', 165569735.59479022)
('bid', 20472124.967617005)

-------------------------------------------
Time: 2019-05-25 06:43:00
-------------------------------------------
('price_volume', 8.08756960289613)

-------------------------------------------
Time: 2019-05-25 06:44:00
-------------------------------------------
('ask', 3104822.9589077695)
('bid', 4846398.026894507)

-------------------------------------------
Time: 2019-05-25 06:44:00
-------------------------------------------
('price_volume', 0.6406454735409526)

-------------------------------------------
Time: 2019-05-25 06:45:00
-------------------------------------------
('ask', 3325494.6286898884)
('bid', 4287219.04786571)

-------------------------------------------
Time: 2019-05-25 06:45:00
-------------------------------------------
('price_volume', 0.7756763980476824)

------------------------------------

-------------------------------------------
Time: 2019-05-25 07:09:00
-------------------------------------------
('ask', 2625355.5005478747)
('bid', 3546814.622271754)

-------------------------------------------
Time: 2019-05-25 07:09:00
-------------------------------------------
('price_volume', 0.7402009352454739)

-------------------------------------------
Time: 2019-05-25 07:10:00
-------------------------------------------
('ask', 2037454.320196367)
('bid', 3995502.1050251955)

-------------------------------------------
Time: 2019-05-25 07:10:00
-------------------------------------------
('price_volume', 0.5099369908061953)

-------------------------------------------
Time: 2019-05-25 07:11:00
-------------------------------------------
('ask', 1557807.205079079)
('bid', 2519956.9306298634)

-------------------------------------------
Time: 2019-05-25 07:11:00
-------------------------------------------
('price_volume', 0.61818802779685)

------------------------------------

-------------------------------------------
Time: 2019-05-25 07:35:00
-------------------------------------------
('ask', 2843884.9431579025)
('bid', 3803101.569304925)

-------------------------------------------
Time: 2019-05-25 07:35:00
-------------------------------------------
('price_volume', 0.7477804342937036)

-------------------------------------------
Time: 2019-05-25 07:36:00
-------------------------------------------
('ask', 3664797.291952905)
('bid', 2904905.6156827426)

-------------------------------------------
Time: 2019-05-25 07:36:00
-------------------------------------------
('price_volume', 1.2615891105610204)

-------------------------------------------
Time: 2019-05-25 07:37:00
-------------------------------------------
('ask', 4286589.880723353)
('bid', 2363872.0745886206)

-------------------------------------------
Time: 2019-05-25 07:37:00
-------------------------------------------
('price_volume', 1.8133764203248344)

----------------------------------

-------------------------------------------
Time: 2019-05-25 08:01:00
-------------------------------------------
('ask', 1456370.7961607052)
('bid', 3408387.5246972837)

-------------------------------------------
Time: 2019-05-25 08:01:00
-------------------------------------------
('price_volume', 0.4272902613355425)

-------------------------------------------
Time: 2019-05-25 08:02:00
-------------------------------------------
('ask', 590077.5102112147)
('bid', 2403800.7490528747)

-------------------------------------------
Time: 2019-05-25 08:02:00
-------------------------------------------
('price_volume', 0.24547688091191086)

-------------------------------------------
Time: 2019-05-25 08:03:00
-------------------------------------------
('ask', 1240882.4140252497)
('bid', 3735830.6537542767)

-------------------------------------------
Time: 2019-05-25 08:03:00
-------------------------------------------
('price_volume', 0.33215702986382434)

------------------------------

In [25]:
ssc.stop()